In [10]:
! pip install pynini

In [13]:
def n_vowel_categorize(lemma):
  for char in reversed(lemma):
    if char in "öü":
      return {"frontness":True, "roundness":True}
    elif char in "ei":
      return {"frontness":True, "roundness":False}
    elif char in "ou":
      return {"frontness":False, "roundness":True}
    elif char in "aı":
      return {"frontness":False, "roundness":False}

def v_vowel_categorize(lemma):
  vowels = "öüeiouaı"
  n = 0
  for char in reversed(lemma):
    if char in vowels:
      n += 1
      if n == 2:
        if char in "öü":
          return {"frontness": True, "roundness": True}
        elif char in "ei":
          return {"frontness": True, "roundness": False}
        elif char in "ou":
          return {"frontness": False, "roundness": True}
        elif char in "aı":
         return {"frontness": False, "roundness": False}

def is_str(msd):
  return "ACC" in msd or "GEN" in msd

def is_obl(msd):
  return "DAT" in msd or "LOC" in msd or "ABL" in msd

with open('data/tur.out', 'r') as file:
    tur_out = [line.strip().split('\t') for line in file if line.strip()]

with open('data/tur.dev', 'r') as file:
    tur_dev = [line.strip().split('\t') for line in file if line.strip()]

dev = []
for word in tur_dev:
  dev.append(word[2])

In [14]:
# this is for correcting noun plural suffix
import pynini

# Define Turkish alphabet
turkish_alphabet = "abcçdefgğhıijklmnoöprsştuüvyz " # add " "(space), so it includes word compounds, like "sözdizimsel tuzlerde" etc.
sigma = pynini.union(*turkish_alphabet).closure()

# Plural suffix context for a
plural_correction_rule_a = pynini.cdrewrite(
        pynini.cross("e", "a"),
        "l",
        "r",
        sigma)

# Plural suffix context for e
plural_correction_rule_e = pynini.cdrewrite(
        pynini.cross("a", "e"),
        "l",
        "r",
        sigma)

n_n_pl = 0
c_n_pl = 0

# Process
for i, line in enumerate(tur_out):
  lemma, msd, inflected = line[:3]
  correct = dev[i]

  if msd.startswith("N") and '(PL' in msd:
    v_cat = n_vowel_categorize(lemma)

    if v_cat["frontness"] == False and "ler" in inflected:
      output = inflected @ plural_correction_rule_a
      paths = list(output.paths().ostrings())
      correction = paths[0]
      print(lemma, inflected, "→", correction, correct == correction)
      n_n_pl += 1
      if correct == correction:
        c_n_pl += 1

    elif v_cat["frontness"] == True and "lar" in inflected:
      output = inflected @ plural_correction_rule_e
      paths = list(output.paths().ostrings())
      correction = paths[0]
      print(lemma, inflected, "→", correction, correct == correction)
      n_n_pl += 1
      if correct == correction:
        c_n_pl += 1

    else:
      correction = inflected

print("--------------------------------")
print("Number of words identified:", n_n_pl)
print("Number of words corrected:", c_n_pl)

anahtarlık anahtarlıklerimize → anahtarlıklarimize False
asimptot asimptotlerimizden → asimptotlarimizden False
asimptot asimptotlerinden → asimptotlarinden False
asimptot asimptotlerinden → asimptotlarinden False
asimptot asimptotlerimizi → asimptotlarimizi False
asimptot asimptotlerini → asimptotlarini False
asimptot asimptotlerini → asimptotlarini False
asimptot asimptotlerimize → asimptotlarimize False
asimptot asimptotlerine → asimptotlarine False
asimptot asimptotlerimizin → asimptotlarimizin False
asimptot asimptotlerinin → asimptotlarinin False
asimptot asimptotlerinde → asimptotlarinde False
asimptot asimptotlerimiz → asimptotlarimiz False
asimptot asimptotleriniz → asimptotlariniz False
küçüklük küçüklükların → küçüklüklerın False
küçüklük küçüklüklar → küçüklükler True
otel otellarını → otellerını False
otel otellarının → otellerının False
sözdizimsel tuz sözdizimsel tuzleri → sözdizimsel tuzlari False
sözdizimsel tuz sözdizimsel tuzlerin → sözdizimsel tuzlarin False
sözdizi

In [15]:
# this is for correcting & adding missing vowel in verb progressive suffix
import pynini

# Define Turkish alphabet
turkish_alphabet = "abcçdefgğhıijklmnoöprsştuüvyzT " # add " "(space) and T, so it includes word compounds and capital letter
sigma = pynini.union(*turkish_alphabet).closure()
vowel = pynini.union(*"öüeiouaı") # define vowels group
consonant = pynini.union(*"bcçdfgğhjklmnprsştvz")  # define consonants group

# Progressive suffix context for adding ü
prog_adding_rule_ü = pynini.cdrewrite(
      pynini.cross("", "ü"),
      consonant,    # Left context: a consonant
      "yor",    # Right context: before "yor"
      sigma)

# Progressive suffix context for adding i
prog_adding_rule_i = pynini.cdrewrite(
      pynini.cross("", "i"),
      consonant,    # Left context: a consonant
      "yor",    # Right context: before "yor"
      sigma)

# Progressive suffix context for correcting back to ü
prog_correction_rule_ü = pynini.cdrewrite(
        pynini.cross(vowel, "ü"),
        "",    # Left context: anything
        "yor",    # Right context: before "yor" (equal to immediately followed by 'yor')
        sigma)

# Progressive suffix context for correcting back to i
prog_correction_rule_i = pynini.cdrewrite(
        pynini.cross(vowel, "i"),
        "",    # Left context: anything
        "yor",    # Right context: before "yor" (equal to immediately followed by 'yor')
       sigma)

n_v_prog = 0
c_v_prog = 0

# Process
for i, line in enumerate(tur_out):
  lemma, msd, inflected = line[:3]
  correct = dev[i]

  if msd.startswith("V") and 'PROG' in msd:
    v_cat = v_vowel_categorize(lemma)
    # first, adding missing vowel ü, then correcting
    if v_cat["frontness"] == True and v_cat["roundness"] == True:
      output = inflected @ prog_adding_rule_ü
      output = output @ prog_correction_rule_ü
      paths = list(output.paths().ostrings())
      correction = paths[0]
      print(lemma, inflected, "→", correction, correct == correction)
      n_v_prog += 1
      if correct == correction:
        c_v_prog += 1

    # first, adding missing vowel i, then correcting
    elif v_cat["frontness"] == True and v_cat["roundness"] == False:
      output = inflected @ prog_adding_rule_i
      output = output @ prog_correction_rule_i
      paths = list(output.paths().ostrings())
      correction = paths[0]
      print(lemma, inflected, "→", correction, correct == correction)
      n_v_prog += 1
      if correct == correction:
        c_v_prog += 1

    else:
      correction = inflected

print("--------------------------------")
print("Number of words identified:", n_v_prog)
print("Number of words corrected:", c_v_prog)

Türkçeleştirmek Türkçeleştirüyor muydu → Türkçeleştiriyor muydu True
Türkçeleştirmek Türkçeleştirmiyormuşuz → Türkçeleştirmiyormuşuz True
Türkçeleştirmek Türkçeleştiriyormuş → Türkçeleştiriyormuş True
Türkçeleştirmek Türkçeleştiryor olmamalıymışım → Türkçeleştiriyor olmamalıymışım True
Türkçeleştirmek Türkçeleştiriyor olmalıymışsın → Türkçeleştiriyor olmalıymışsın True
Türkçeleştirmek Türkçeleştiriyor olmalı mıydım → Türkçeleştiriyor olmalı mıydım True
Türkçeleştirmek Türkçeleştiryor olmalıydım → Türkçeleştiriyor olmalıydım True
bitirmek bitiriyor olmayacağız → bitiriyor olmayacağız True
bitirmek bitiriyor olmayacaksın → bitiriyor olmayacaksın True
bitirmek bitiryor olur muydu → bitiriyor olur muydu True
bitirmek bitirüyor muydu → bitiriyor muydu True
bitirmek bitirmiyordu → bitirmiyordu True
bitirmek bitiryor olacak mıydım → bitiriyor olacak mıydım True
bitirmek bitiriyor olmayacakmışım → bitiriyor olmayacakmışım True
büktürmek büktürüyor olurlardı → büktürüyor olurlardı True
büktürme